In [53]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from collections import namedtuple

# $\Delta t$

In [ ]:
duration = 10
delta_t = 0.1
time_series = np.arange(0, duration, delta_t)
# neuron.active(t_index) = neuron.u.update(neuron[t_index -1], config)
I = np.random.random(time_series.shape) * (3.5 - 3) + 3

### problems
- [ ] Connect for Multi pops
- [ ] Reseting neurons

### FX
- [ ] U update in itegration (Series)
- [ ] update connection for input connection for next layer neurons



# Models/LIF

- [ ] API
- [ ] parameters importance 
- [ ] How it works simple-introductions

In [ ]:
class LIF:
    __slots__ = ['dt__τ', 'θ', 'R', 'u_rest', 'u', 'input',  'spike_trace', 'isi', 'posts', 'input']
 
    def __init__(self, config: namedtuple):
        self.θ = config.threshold
        self.R = config.resistor
        self.u_rest = config.uRest
        self.dt = config.dt 
        self.τ = config.tau
        self.isi = int(config.isInhibitory)
        self.u = self.u_rest
        self.input = {}
        self.spike_trace = []
        self.posts = []
        
    def integrate(self, I, t):
        """ 
        integrate(currentValue, currentTime)
        return one if neuron spikes in currentTime otherwise it will return zero
        """
        
        # compute potential
        self.u += (self.u_rest - self.u + self.R * I) * (self.dt / self.τ)
        # Add presynaptic input
        self.u += self.input.get(t, 0) * self.R * (self.dt / self.τ)
        #  compute spikes
        if self.u >= self.θ:
            self.u = self.u_rest
            self.spike_trace.append(t)
            # update entry input for post synaptic neurons
            for synapse in self.posts:
                fs_time = t + (1//self.dt*self.dt) + self.dt  # future spikes time
                if fs_time no in synpse.neuron.input:
                        synapse.neuron.input[fs_time] = 0
                synapse.neuron.input[fs_time] += pow(-1,self.isi) * synapse.w
        
        return int(t in self.spike_trace[-1:]) # if neuron has just spiked in time t return 1 otherwise 0    
    
    def reset(self, t, alpha):
        # reset inputs
        if self.input.get(t, 0) == 0:
            return
        value = self.input.pop(t)
        # search for next 6 feasable seconds delayed
        for sec in range(1, 6):
            fs_time = t + (sec//self.dt*self.dt) 
            if fs_time in self.input:
                self.input[fs_time] += value * alpha

as Eq. (12.4) says I(t) is independent of the neuronal index i.

$I(t) = w_0 N \int_0^∞ α(s)A(t −s)ds + I_{ext}(t)$

In [163]:
class Neuron:
    """ LIF Neuron """
    __slots__ = ['dt', 'τ', 'θ', 'R', 'u_rest', 'u', 'Ii', 'isi']
    
    def __init__(self, config: namedtuple):
        """ Initialize Configs """
        self.θ      = config.threshold
        self.R      = config.resistor
        self.u_rest = config.uRest
        self.τ      = config.tau
        self.dt     = config.dt
        self.isi    = int(config.isInhibitory)
        self.u      = self.u_rest - 0.1 # this prevent extra count for first spikes count 
        self.Ii    = 0 # internal I value (gained by Hebbian Rules)  
        # self.spike_trace = []
        # self.posts = []

    def integrate(self, Iext):
        """
            Go ahead one step in time windows and  update potential
            Iext := I_ext(t) can be constant or Gaussian uniform number
            I = Iext + generic_timecourse
        """
        spiked = False

        self.u += (self.u_rest - self.u + self.R * (Iext + self.Ii)) * (self.dt / self.τ)
        # Reseting on exciding θ threshhold  
        if(self.u >= self.θ):
            self.u = self.u_rest
            spiked = True

        return (self.u, spiked)
    
    # update Ii
    # Handle for synapse in self.posts:
    #     synapse.neuron.input += pow(-1,self.isi) * synapse.w * (self.θ - self.u_rest)
        

In [194]:
class Connection:
    """ Connect population A to population B with connection_type strategy
valid connection_types
    'full' for  full_connectivity,
    'fixedCP' for fixed_coupling_probability_connectivity,
    'fixedNPP' for fixed_number_of_presynaptics_parents,

For illiminate performance issue it doesn't connect neurons until calling @method for comp
"""
    def __init__(self, A, B, connection_type, **configs):
        """ connect A population to B population with input connection_type """
        self.pre = A.neurons
        self.post = B.neurons
        self.relations = []
        connect = {
            'full', Connection.full_connectivity,
            'fixedCP': Connection.fixed_coupling_probability_connectivity,
            'fixedNPP': Connection.fixed_number_of_presynaptics_parents,
        }.get(connection_type, fixedCP)

        connect(self.pre, self.post, **configs)

    @staticmethod
    def full_connectivity(A, B, p=0.1, J0=10, r0=1): 
        C = p * len(A) # p * N
        μ, σ = J0/C, r0/math.sqrt(C)
        normal = np.random.normal

        for pre in A:
            for post in B:
                relations.append([pre, post, normal(μ, σ)])
    
    @staticmethod
    def fixed_coupling_probability_connectivity(A, B, p=0.1, J0=10, r0=1): 
        """ p := coupling_probability """
        CA_size = int(p * len(A))
        CB_size = int(p * len(B))
        μ, σ = J0/CB_size, r0/math.sqrt(CB_size)
        normal = np.random.normal

        for pre in np.random.choice(A, CA_size, replace=False):
            for post in np.random.choice(B, CB_size, replace=False)
                relations.append([pre, post, normal(μ, σ)])

    
    @staticmethod
    def fixed_number_of_presynaptics_parents(A, B, p=0.1, J0=10, r0=1): 
        C = int(p * len(A))
        μ, σ = J0/C, r0/math.sqrt(C)
        normal = np.random.normal
        
        for post in B:
            for pre in np.random.choice(A, C, replace=False):
                relations.append([pre, post, normal(μ, σ)])

    def update():
        """ 
            For every post of spiked neuron you should update Ii 
            For stdp learning it should update Ws
        """
        pass

SyntaxError: invalid syntax (<ipython-input-194-adab6a33fdf4>, line 17)

In [ ]:
time_series = np.ones((35, 4))

In [ ]:
def get_homogenous_group():
    pass

In [ ]:
class Learning:
    @staticmethod
    def stdp(w, t_pre, t_post, **config):
        # deafult values
        A__    = config.get('A__', lambda x: 10)
        A_     = config.get('A_', lambda x: -10)
        tau__  = config.get('tau__', 5)
        tau_   = config.get('tau_', 5)
        
        Δt =  t_post - t_pre
        if (Δt >= 0):
            Δw =  A__(w) * np.exp(-abs(Δt) / tau__)    
        else:
            Δw =  A_(w) * np.exp(-abs(Δt) / tau_)
        return Δw, Δt 

In [135]:
class Configuration:
    lif = namedtuple('LIFConfig', 'tau resistor threshold uRest dt isInhibitory')
    # μ = 0
    # σ = 1

    @classmethod
    def lif_config_generator(cls, dt, isInhibitory):
        
        return lambda _: cls.lif(**{
            # fixed for all neurons
            "tau" :5, 
            "resistor": 5,
            "threshold": -65,
            "uRest": -70,
            "dt": dt, 
            "isInhibitory": isInhibitory, 
        })

In [178]:
class Population:
    """ Gonna Handle homogenous population """
    def __init__(self, neuron, config_generator ,**configs):
        dt = configs.get('time_step', 0.1) 
        duration = configs.get('duration', 5)        
        self.time_series = np.arange(0,duration,dt)
        self.size = configs.get('size', 1000)

        self.potentials_series = np.empty((self.size, self.time_series.size))
        
        # Neurons (homogenous)
        self.neurons = [
            neuron(config_generator(i)) for i in range(self.size)
        ]

    def yeChizi(self):
        pass
    
    def activate(self, t):
        """ Activate a population """
        assert 0 <= t < self.time_series.size, 'Please Enter an integer belongs to [0, duration/dt) range as an index.'
        # TODO: need I_ext calculation from static fixed I ~ N(μ, σ), or separated for each individual
        I_ext = 35
        potentials = np.fromiter(
            (n.integrate(I_ext) for (index, n) in enumerate(self.neurons)),
            np.dtype([('u', 'f'), ('spiked','?')])
        )
        self.potentials_series[:, t] = potentials['u']
        # update interanl I for neurons has spiked in very last time potentials['spiked']
        return potentials

In [179]:
cgen = Configuration.lif_config_generator(0.1, False)
pop = Population(Neuron, cgen, size=10)


In [180]:
p0 = pop.activate(0)

In [182]:
p0['spiked']

array([False, False, False, False, False, False, False, False, False,
       False])

In [186]:
p1= pop.activate(1)

In [188]:
pop.potentials_series[:, 1]

array([-70., -70., -70., -70., -70., -70., -70., -70., -70., -70.])

In [191]:
class A:
    def __init__(self, a):
        A.f(a)

    @staticmethod
    def f(a):
        print(a)

In [192]:
A(5)

5


In [193]:
sqrt(9)

NameError: name 'sqrt' is not defined